In [17]:
import pandas as pd
import numpy as np
import os
import datetime as dt
import scipy.stats as stats
from scipy.interpolate import interp1d
import datetime as dt
import sys
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from SNOTEL_splitter import splitFireSNOTEL

import seaborn as sns
import statsmodels.formula.api as smf
%matplotlib inline

In [18]:
SMALL_SIZE = 10
MEDIUM_SIZE = 12
BIGGER_SIZE = 14

plt.rc('font', size=MEDIUM_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

In [19]:
# fire locations
fire_sites = {'Sentinel Butte':'Gold Axe Camp', # apparent postive/null
              'Grouse Camp':'Trough', # apparent negative
              'Harts Pass':'Lyman Lake', # apparent positive/null
              'Three Creeks Meadow': 'Salt Creek Falls', # apparent negative/null
              'Pope Ridge': 'Blewett Pass'}# apparent negative/null
filepath = r'C:\Users\dlhogan\OneDrive - UW\Documents\GitHub\CEWA565_project\data\fire_sites'
disruption_dates = pd.read_excel(os.path.join(filepath,'snotel_fire_data_new.xlsx'))
variable = 'swe'

In [20]:
def getSites(site, paired, fire_dict=fire_sites, filepath=filepath, disruption_dates=disruption_dates, variable=variable,postyrs=5):
    """getSites will return filtered dataframes for the inputed site and its paired site, and can filter 
       to particular number of years after the disturbance

    Args:
        site (str): site of interest
        paired (str): paired site
        fire_dict (dict): dict containing sites and paired sites. Defaults to fire_sites.
        filepath (str): Path to data location. Defaults to filepath.
        disruption_dates (df): df containing fire start date and intensity values. Defaults to disruption_dates.
        variable (str): swe or depth. Can be adapted set of variables of interest. Defaults to swe.
        postyrs (int, optional): number of years after distrubance to report data. Defaults to 5.

    Returns:
        [type]: [description]
    """
    melt=False
    if variable == 'meltdates':
        variable = 'swe_meltdates'
        melt = True
    df_b, df_a = splitFireSNOTEL(site,fire_sites,variable,disruption_dates,filepath)
    df_paired_b, df_paired_a = splitFireSNOTEL(paired,fire_sites,variable,disruption_dates,filepath)
    print(df_paired_a)
    if melt is False:
        fire_year = min(df_a.index.year)
        if postyrs != 'max':
            df_a = df_a.loc[dt.date(fire_year,10,1):dt.date(fire_year+postyrs,9,30)]
            df_paired_a = df_paired_a.loc[dt.date(fire_year,10,1):dt.date(fire_year+postyrs,9,30)]
        # remove 2015 as exceptionally dry year
        for df in [df_b,df_a,df_paired_a,df_paired_b]:
            df = df[df.index.year != 2015]
            df = df.dropna()
            print(df)
        if len(df_paired_b) > len(df_b):
            min_date = min(df_b.index)
            df_paired_b = df_paired_b.loc[df_paired_b.index > min_date]
        elif len(df_b) > len(df_paired_b):
            min_date = min(df_paired_b.index)
            df_b = df_b.loc[df_b.index > min_date]
    else:
        if postyrs != 'max':
            df_a = df_a[df_a['water_year'] < (min(df_a['water_year']+postyrs))]
            df_paired_a = df_paired_a[df_paired_a['water_year'] < (min(df_paired_a['water_year']+postyrs))]
        # remove 2015 as exceptionally dry year
        
        if len(df_paired_b) > len(df_b):
            min_date = min(df_b['water_year'])
            df_paired_b = df_paired_b.loc[df_paired_b['water_year'] > min_date]
        elif len(df_b) > len(df_paired_b):
            min_date = min(df_paired_b['water_year'])
            df_b = df_b.loc[df_b['water_year'] > min_date]
        for df in [df_b,df_a,df_paired_a,df_paired_b]:
            df = df[df['water_year']!= 2015].set_index('water_year')
    print('Filtering to {} years after fire...'.format(postyrs))
    print('Done!')
    return df_b, df_a, df_paired_b, df_paired_a

In [9]:
def max_swe(df):
    df = df.dropna()
    max_swe = df.loc[df.groupby('water_year')['SNOTEL:SNWD_D'].idxmax()].reset_index().set_index('water_year')#df[(df.index.month==4) & (df.index.day==1)].reset_index().set_index('water_year')
    dowy = []
    for date in max_swe['datetime']:
        dowy.append(date2DOWY(date))
    max_swe['dowy_max'] = dowy
    max_swe = max_swe[['dowy_max','SNOTEL:SNWD_D']]
    max_swe = max_swe[max_swe['dowy_max'].between(122,250)]
    return max_swe

In [10]:
def date2DOWY(day_of_interest):
    day_of_interest = day_of_interest.date()
    dowy_start = dt.date(day_of_interest.year-1,10,1)
    dowy = day_of_interest - dowy_start
    return dowy.days

In [7]:
diff_in_diff_results = {}
for i,(site,paired) in enumerate(fire_sites.items()):
    df_b, df_a, df_paired_b, df_paired_a = getSites(site,paired, variable='depth',postyrs=5)
    for df in [df_b, df_a, df_paired_b, df_paired_a]:
        df['water_year'] = df.index.year
    df_b_max = max_swe(df_b)
    df_a_max = max_swe(df_a) 
    df_paired_b_max = max_swe(df_paired_b) 
    df_paired_a_max = max_swe(df_paired_a) 
    
    df_b_melt, df_a_melt, df_paired_b_melt, df_paired_a_melt = getSites(site,paired, variable='meltdates',postyrs=5)
    
        
    df_join = df_b_max.join(df_b_melt.set_index('water_year',drop=False))
    df_join['melt_rate'] = df_join['SNOTEL:SNWD_D']/(df_join['DOWY']-df_join['dowy_max'])
    df1 = df_join
    df1 = df1[['water_year','melt_rate']].dropna()
    df1['pre_post'] = 0
    df1['original'] = 1

    df_join = df_paired_b_max.join(df_paired_b_melt.set_index('water_year',drop=False))
    df_join['melt_rate'] = df_join['SNOTEL:SNWD_D']/(df_join['DOWY']-df_join['dowy_max'])
    df2 = df_join
    df2 = df2[['water_year','melt_rate']].dropna()
    df2['pre_post'] = 0
    df2['original'] = 0

    df_join = df_a_max.join(df_a_melt.set_index('water_year',drop=False))
    df_join['melt_rate'] = df_join['SNOTEL:SNWD_D']/(df_join['DOWY']-df_join['dowy_max'])
    df3 = df_join
    df3 = df3[['water_year','melt_rate']].dropna()
    df3['pre_post'] = 1
    df3['original'] = 1

    df_join = df_paired_a_max.join(df_paired_a_melt.set_index('water_year',drop=False))
    df_join['melt_rate'] = df_join['SNOTEL:SNWD_D']/(df_join['DOWY']-df_join['dowy_max'])
    df4 = df_join
    df4 = df4[['water_year','melt_rate']].dropna()
    df4['pre_post'] = 1
    df4['original'] = 0

    df = pd.concat([df1,df2,df3,df4])

    results = smf.ols("melt_rate ~ original*pre_post",data=df).fit()
    print(results.summary())
    # if i == 2:
    #     sm.graphics.plot_partregress(results, 'original:pre_post')
    results_as_html = results.summary().tables[1].as_html()

    diff_in_diff_ols = pd.read_html(results_as_html,header=0, index_col=0)[0]
    diff_in_diff_ols.to_excel('data/meltrate/'+site+'_diff_in_diff_rate.xlsx')
    diff_in_diff_results[site] = [diff_in_diff_ols,results]
    post_fire_difference = diff_in_diff_ols.loc['original:pre_post']['coef']
    post_fire_stderror = diff_in_diff_ols.loc['original:pre_post']['std err']

    print('Post fire influence on melt rate at {}: {} cm/day'.format(site,post_fire_difference))
    print('Post fire influence on melt rate + standard error: {} cm/day'.format(post_fire_difference+post_fire_stderror))
    print('Post fire influence on melt rate - standard error: {} cm/day'.format(post_fire_difference-post_fire_stderror))

Sentinel Butte for depth
Sentinel Butte for depth
            SNOTEL:SNWD_D qualifiers censor_code        date_time_utc  \
datetime                                                                
2010-10-01            0.0          E          nc  2010-10-01T00:00:00   
2010-10-02            0.0          E          nc  2010-10-02T00:00:00   
2010-10-03            0.0          E          nc  2010-10-03T00:00:00   
2010-10-04            0.0          E          nc  2010-10-04T00:00:00   
2010-10-05            0.0          E          nc  2010-10-05T00:00:00   
...                   ...        ...         ...                  ...   
2015-08-06            0.0          E          nc  2015-08-06T00:00:00   
2015-08-07            0.0          V          nc  2015-08-07T00:00:00   
2015-08-08            0.0          E          nc  2015-08-08T00:00:00   
2015-08-09            0.0          E          nc  2015-08-09T00:00:00   
2015-08-10            0.0          E          nc  2015-08-10T00:00:00   



C:\Users\dlhogan\miniconda3\envs\downgrade\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
C:\Users\dlhogan\miniconda3\envs\downgrade\lib\site-packages\scipy\stats\stats.py:1541: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=16
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


                            OLS Regression Results                            
Dep. Variable:              melt_rate   R-squared:                       0.184
Model:                            OLS   Adj. R-squared:                 -0.020
Method:                 Least Squares   F-statistic:                    0.8996
Date:                Wed, 01 Dec 2021   Prob (F-statistic):              0.470
Time:                        12:01:57   Log-Likelihood:               0.018321
No. Observations:                  16   AIC:                             7.963
Df Residuals:                      12   BIC:                             11.05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.8194      0.16

ValueError: min() arg is an empty sequence

In [ ]:
df_join = df_a_max.join(df_a_melt.set_index('water_year'))
df_join['melt_rate'] = df_join['DOWY']-df_join['dowy_max']

In [21]:
df_b_melt, df_a_melt, df_paired_b_melt, df_paired_a_melt = getSites('Harts Pass','Lyman Lake', variable='depth',postyrs=5)

Harts Pass for depth
Harts Pass for depth
            SNOTEL:SNWD_D qualifiers censor_code        date_time_utc  \
datetime                                                                
2005-09-15            0.0          E          nc  2005-09-15T00:00:00   
2005-09-16            0.0          E          nc  2005-09-16T00:00:00   
2005-09-17            0.0          E          nc  2005-09-17T00:00:00   
2005-09-18            0.0          E          nc  2005-09-18T00:00:00   
2005-09-19            0.0          E          nc  2005-09-19T00:00:00   
...                   ...        ...         ...                  ...   
2021-11-10            NaN        NaN          nc  2021-11-10T00:00:00   
2021-11-11            NaN        NaN          nc  2021-11-11T00:00:00   
2021-11-12            NaN        NaN          nc  2021-11-12T00:00:00   
2021-11-13            NaN        NaN          nc  2021-11-13T00:00:00   
2021-11-14           46.0          E          nc  2021-11-14T00:00:00   

        

ValueError: min() arg is an empty sequence